In [3]:
data = {
    'location': 'Grad Zagreb, Trešnjevka - Jug, Jarun',
    'make': 'Mercedes-Benz',
    'model': 'C-klasa',
    'type': '220 D',
    'chassis_number': 'W1K2060161F006816',
    'manufacture_year': '2021. godište',
    'model_year': '2018.',
    'in_traffic_since': '2021.',
    'mileage': '84000 km',
    'engine': 'Diesel',
    'power': '143 kW',
    'displacement': '1.950 cm3',
    'transmission': 'Automatski sekvencijski',
    'fuel_consumption': '23.7 l/100km',
    'average_CO2_emission': '131.1 g/km',
    'eco_category': 'Euro 6',
    'condition': 'rabljeno',
    'owner': 'prvi',
    'service_book': 'Da',
}

In [13]:
def transform_data(data):
    transformations = {
        "manufacture_year": lambda x: int(x.split(".")[0])
        if "." in x
        else int(x),
        "model_year": lambda x: int(x.split(".")[0]) if "." in x else int(x),
        "mileage": lambda x: int(x.split()[0].replace(".", "")),
        "power": lambda x: int(x.split()[0]),
        "service_book": lambda x: x.lower() == "da",
        "fuel_consumption": lambda x: float(x.split()[0].replace(",", ".")),
        "average_CO2_emission": lambda x: float(
            x.split()[0].replace(",", ".")
        ),
        "owner": lambda x: int(x.split()[0]) if x.split()[0].isdigit() else x,
        "displacement": lambda x: int(x.replace(".", "").replace(" cm3", "")),
        "in_traffic_since": lambda x: int(x.split(".")[0])
        if "." in x
        else int(x),
    }

    transformed_data = {}
    for key, value in data.items():
        if key in transformations:
            try:
                transformed_data[key] = transformations[key](value)
            except Exception as e:
                print(f"Error transforming {key}: {e}")
                transformed_data[key] = value  # fallback to original value
        else:
            transformed_data[key] = value  # no transformation needed

    return transformed_data


In [18]:
transform_data(data)

{'location': 'Grad Zagreb, Trešnjevka - Jug, Jarun',
 'make': 'Mercedes-Benz',
 'model': 'C-klasa',
 'type': '220 D',
 'chassis_number': 'W1K2060161F006816',
 'manufacture_year': 2021,
 'model_year': 2018,
 'in_traffic_since': 2021,
 'mileage': 84000,
 'engine': 'Diesel',
 'power': 143,
 'displacement': 1950,
 'transmission': 'Automatski sekvencijski',
 'fuel_consumption': 23.7,
 'average_CO2_emission': 131.1,
 'eco_category': 'Euro 6',
 'condition': 'rabljeno',
 'owner': 'prvi',
 'service_book': True}

In [27]:
import psycopg2
from datetime import datetime

In [29]:
conn = psycopg2.connect(
    dbname="ads_db",  # Default database
    user="adsuser",
    password="pass",
    host="localhost",  # or your server IP
    port="5432",  # Default PostgreSQL port
)
cursor = conn.cursor()

transformed_data = transform_data(data)
print(transformed_data)

now = datetime.now().isoformat()
transformed_data["date_created"] = now

columns = ", ".join(transformed_data.keys())
values = ", ".join([f"%({key})s" for key in transformed_data.keys()])

# Define the SQL query to insert the data
insert_query = f"""
INSERT INTO ads ({columns})
VALUES ({values});
"""

cursor.execute(insert_query, transformed_data)
conn.commit()

cursor.close()
conn.close()


{'location': 'Grad Zagreb, Trešnjevka - Jug, Jarun', 'make': 'Mercedes-Benz', 'model': 'C-klasa', 'type': '220 D', 'chassis_number': 'W1K2060161F006816', 'manufacture_year': 2021, 'model_year': 2018, 'in_traffic_since': 2021, 'mileage': 84000, 'engine': 'Diesel', 'power': 143, 'displacement': 1950, 'transmission': 'Automatski sekvencijski', 'fuel_consumption': 23.7, 'average_CO2_emission': 131.1, 'eco_category': 'Euro 6', 'condition': 'rabljeno', 'owner': 'prvi', 'service_book': True}
